In [136]:
import pandas as pd

import numpy as np

In [137]:
df = pd.read_csv("./DOHMH_New_York_City_Restaurant_Inspection_Results.csv")
df.head()

,DBA,BORO,CUISINE DESCRIPTION,VIOLATION CODE,SCORE,GRADE,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,17.0,B,40.829178,-73.875707
1,ARIANA KEBAB HOUSE,Manhattan,Afghan,10B,5.0,A,40.764993,-73.987856
2,TAP NYC,Manhattan,Café/Coffee/Tea,08C,20.0,NaN,40.777716,-73.978556
3,GEORGES RESTAURANT,Brooklyn,American,10F,7.0,A,40.641088,-74.014474
4,Q & I BAKERY,Brooklyn,Bakery,06C,22.0,NaN,40.628482,-74.006099


In [138]:
df.shape


(397490, 8)

In [139]:
df.dtypes

DBA                     object
BORO                    object
CUISINE DESCRIPTION     object
VIOLATION CODE          object
SCORE                  float64
GRADE                   object
Latitude               float64
Longitude              float64
dtype: object

In [140]:
df.columns

Index(['DBA', 'BORO', 'CUISINE DESCRIPTION', 'VIOLATION CODE', 'SCORE',
       'GRADE', 'Latitude', 'Longitude'],
      dtype='object')

In [141]:
df = df.rename(columns={"DBA":"Name","BORO":"Borough", "CUISINE DESCRIPTION":"Cuisine", "VIOLATION CODE":"Violation Code", "SCORE":"Score", "GRADE":"Grade"})

df.columns

Index(['Name', 'Borough', 'Cuisine', 'Violation Code', 'Score', 'Grade',
       'Latitude', 'Longitude'],
      dtype='object')

In [142]:
df = df.dropna(subset=['Name', 'Violation Code','Score', 'Grade'])
df.head()

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,17.0,B,40.829178,-73.875707
1,ARIANA KEBAB HOUSE,Manhattan,Afghan,10B,5.0,A,40.764993,-73.987856
3,GEORGES RESTAURANT,Brooklyn,American,10F,7.0,A,40.641088,-74.014474
7,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,9.0,A,40.704818,-73.934015
8,"DUNKIN', BASKIN ROBBINS",Manhattan,Donuts,10B,9.0,A,40.750971,-73.996492


In [143]:
df.shape

(201337, 8)

In [144]:
df = df[df.Latitude != 0]
df = df[df.Longitude != 0]
df = df[df.Grade != "N"]
df = df[df.Grade != "G"]
df = df[df.Grade != "Z"]
df.shape

(193344, 8)

In [145]:
df.head()

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,17.0,B,40.829178,-73.875707
1,ARIANA KEBAB HOUSE,Manhattan,Afghan,10B,5.0,A,40.764993,-73.987856
3,GEORGES RESTAURANT,Brooklyn,American,10F,7.0,A,40.641088,-74.014474
7,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,9.0,A,40.704818,-73.934015
8,"DUNKIN', BASKIN ROBBINS",Manhattan,Donuts,10B,9.0,A,40.750971,-73.996492


In [146]:
codedf = pd.read_csv("./ViolationCodes.csv")
codedf.head()

,Violation Code,Violation Group,Violation Description,Maximum Penalty
0,02A,2,Food temperature storage violation,600
1,02B,2,Food temperature storage violation,600
2,02C,2,Food temperature storage violation,300
3,02D,2,Food temperature storage violation,300
4,02E,2,Food temperature storage violation,200


In [147]:
codedf.set_index('Violation Code')

,Violation Group,Violation Description,Maximum Penalty
Violation Code,,,
02A,2,Food temperature storage violation,600
02B,2,Food temperature storage violation,600
02C,2,Food temperature storage violation,300
02D,2,Food temperature storage violation,300
02E,2,Food temperature storage violation,200
...,...,...,...
22B,22,Failure to take reasonable precautions to prot...,200
22C,22,Failure to take reasonable precautions to prot...,200
22E,22,Failure to take reasonable precautions to prot...,200


In [148]:
mergeddf = pd.merge(df, codedf, on='Violation Code')
mergeddf

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,17.0,B,40.829178,-73.875707,10,Facility and machinery related,200
1,GEORGES RESTAURANT,Brooklyn,American,10F,7.0,A,40.641088,-74.014474,10,Facility and machinery related,200
2,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,9.0,A,40.704818,-73.934015,10,Facility and machinery related,200
3,THAI BKK,Manhattan,Thai,10F,23.0,B,40.802967,-73.938432,10,Facility and machinery related,200
4,MORNING GLORY BAKERY,Queens,Bakery,10F,12.0,A,40.728130,-73.863607,10,Facility and machinery related,200
...,...,...,...,...,...,...,...,...,...,...,...
193339,HA BO KITCHEN,Queens,Chinese,03G,12.0,A,40.713864,-73.735721,3,Food not from an approved source,300
193340,FOUR SEASONS HOTEL,Manhattan,American,03G,11.0,A,40.762051,-73.971793,3,Food not from an approved source,300
193341,NEW YORK STOCK EXCHANGE,Manhattan,American,04I,12.0,A,40.707392,-74.011106,4,"Food contamination with other foods, people, o...",300
193342,DEBORAH MILLER,Manhattan,American,06I,22.0,B,40.711718,-74.000606,6,Food worker or worker cleaning responsibilitie...,350


In [149]:
mergeddf.columns

Index(['Name', 'Borough', 'Cuisine', 'Violation Code', 'Score', 'Grade',
       'Latitude', 'Longitude', 'Violation Group', 'Violation Description',
       'Maximum Penalty'],
      dtype='object')

In [150]:
gradecounts = mergeddf.groupby('Grade').count()
gradecounts

,Name,Borough,Cuisine,Violation Code,Score,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
Grade,,,,,,,,,,
A,156850,156850,156850,156850,156850,156694,156694,156850,156850,156850
B,24837,24837,24837,24837,24837,24819,24819,24837,24837,24837
C,9265,9265,9265,9265,9265,9257,9257,9265,9265,9265
P,2392,2392,2392,2392,2392,2384,2384,2392,2392,2392


In [151]:
groupedbygrade = mergeddf.groupby('Grade')


In [152]:
groupA= groupedbygrade.get_group('A')
groupA.head()

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
1,GEORGES RESTAURANT,Brooklyn,American,10F,7.0,A,40.641088,-74.014474,10,Facility and machinery related,200
2,ROBERTA'S PIZZA & BAKERY,Brooklyn,Pizza,10F,9.0,A,40.704818,-73.934015,10,Facility and machinery related,200
4,MORNING GLORY BAKERY,Queens,Bakery,10F,12.0,A,40.728130,-73.863607,10,Facility and machinery related,200
5,UPTOWN BOURBON,Manhattan,Café/Coffee/Tea,10F,3.0,A,40.829264,-73.948388,10,Facility and machinery related,200
6,LA SABROSURA,Queens,"Latin (Cuban, Dominican, Puerto Rican, South &...",10F,10.0,A,40.766028,-73.919554,10,Facility and machinery related,200


In [153]:
groupA.to_dict

<bound method DataFrame.to_dict of                             Name    Borough  \
1             GEORGES RESTAURANT   Brooklyn   
2       ROBERTA'S PIZZA & BAKERY   Brooklyn   
4           MORNING GLORY BAKERY     Queens   
5                 UPTOWN BOURBON  Manhattan   
6                   LA SABROSURA     Queens   
...                          ...        ...   
193336     GOOD MORNING BROOKLYN   Brooklyn   
193338    ANGIE'S BREAKFAST SPOT   Brooklyn   
193339             HA BO KITCHEN     Queens   
193340        FOUR SEASONS HOTEL  Manhattan   
193341   NEW YORK STOCK EXCHANGE  Manhattan   

                                                  Cuisine Violation Code  \
1                                                American            10F   
2                                                   Pizza            10F   
4                                                  Bakery            10F   
5                                         Café/Coffee/Tea            10F   
6       Latin (Cuban

In [154]:
groupB= groupedbygrade.get_group('B')
groupB.head()

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
0,YANKEE JZ PIZZA,Bronx,Pizza,10F,17.0,B,40.829178,-73.875707,10,Facility and machinery related,200
3,THAI BKK,Manhattan,Thai,10F,23.0,B,40.802967,-73.938432,10,Facility and machinery related,200
11,JR PRIMOS RESTAURANT,Bronx,Spanish,10F,22.0,B,40.865413,-73.867511,10,Facility and machinery related,200
27,CHIU HONG BAKERY,Manhattan,Bakery,10F,24.0,B,40.719573,-73.996281,10,Facility and machinery related,200
36,LAVARTERA MEDITERRANEAN BISTRO,Brooklyn,Mediterranean,10F,20.0,B,40.691398,-73.990794,10,Facility and machinery related,200


In [155]:
groupA.to_dict

<bound method DataFrame.to_dict of                             Name    Borough  \
1             GEORGES RESTAURANT   Brooklyn   
2       ROBERTA'S PIZZA & BAKERY   Brooklyn   
4           MORNING GLORY BAKERY     Queens   
5                 UPTOWN BOURBON  Manhattan   
6                   LA SABROSURA     Queens   
...                          ...        ...   
193336     GOOD MORNING BROOKLYN   Brooklyn   
193338    ANGIE'S BREAKFAST SPOT   Brooklyn   
193339             HA BO KITCHEN     Queens   
193340        FOUR SEASONS HOTEL  Manhattan   
193341   NEW YORK STOCK EXCHANGE  Manhattan   

                                                  Cuisine Violation Code  \
1                                                American            10F   
2                                                   Pizza            10F   
4                                                  Bakery            10F   
5                                         Café/Coffee/Tea            10F   
6       Latin (Cuban

In [156]:
groupC= groupedbygrade.get_group('C')
groupC.head()

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
26,EIGHT JANE FOOD,Queens,Asian,10F,40.0,C,40.761359,-73.831040,10,Facility and machinery related,200
174,CHINA XIANG,Manhattan,Chinese,10F,7.0,C,40.757912,-73.991442,10,Facility and machinery related,200
200,SOLDIERS CARIBBEAN BUFFET RESTAURANT,Brooklyn,Caribbean,10F,3.0,C,40.670561,-73.930545,10,Facility and machinery related,200
210,SUPER BOWL CHINESE RESTAURANT,Queens,Chinese,10F,6.0,C,40.726669,-73.907096,10,Facility and machinery related,200
213,GIGI'S PIZZA & PASTA,Bronx,Pizza/Italian,10F,4.0,C,40.853853,-73.843438,10,Facility and machinery related,200


In [157]:
groupA.to_dict

<bound method DataFrame.to_dict of                             Name    Borough  \
1             GEORGES RESTAURANT   Brooklyn   
2       ROBERTA'S PIZZA & BAKERY   Brooklyn   
4           MORNING GLORY BAKERY     Queens   
5                 UPTOWN BOURBON  Manhattan   
6                   LA SABROSURA     Queens   
...                          ...        ...   
193336     GOOD MORNING BROOKLYN   Brooklyn   
193338    ANGIE'S BREAKFAST SPOT   Brooklyn   
193339             HA BO KITCHEN     Queens   
193340        FOUR SEASONS HOTEL  Manhattan   
193341   NEW YORK STOCK EXCHANGE  Manhattan   

                                                  Cuisine Violation Code  \
1                                                American            10F   
2                                                   Pizza            10F   
4                                                  Bakery            10F   
5                                         Café/Coffee/Tea            10F   
6       Latin (Cuban

In [158]:
groupP= groupedbygrade.get_group('P')
groupP.head()

,Name,Borough,Cuisine,Violation Code,Score,Grade,Latitude,Longitude,Violation Group,Violation Description,Maximum Penalty
13,TASTY THAI,Queens,Thai,10F,7.0,P,40.699992,-73.899383,10,Facility and machinery related,200
21,GOLDEN PUNJAB INDIAN RESTAURANT,Queens,Indian,10F,11.0,P,40.692517,-73.816913,10,Facility and machinery related,200
57,AL HORNO LEAN MEXICAN KITCHEN,Manhattan,Mexican,10F,2.0,P,40.724911,-73.987254,10,Facility and machinery related,200
83,KOREAN NOODLE HOUSE,Queens,Korean,10F,8.0,P,40.759449,-73.772734,10,Facility and machinery related,200
89,MALA PROJECT,Manhattan,Chinese,10F,3.0,P,40.727181,-73.985572,10,Facility and machinery related,200


In [159]:
groupA.to_dict

<bound method DataFrame.to_dict of                             Name    Borough  \
1             GEORGES RESTAURANT   Brooklyn   
2       ROBERTA'S PIZZA & BAKERY   Brooklyn   
4           MORNING GLORY BAKERY     Queens   
5                 UPTOWN BOURBON  Manhattan   
6                   LA SABROSURA     Queens   
...                          ...        ...   
193336     GOOD MORNING BROOKLYN   Brooklyn   
193338    ANGIE'S BREAKFAST SPOT   Brooklyn   
193339             HA BO KITCHEN     Queens   
193340        FOUR SEASONS HOTEL  Manhattan   
193341   NEW YORK STOCK EXCHANGE  Manhattan   

                                                  Cuisine Violation Code  \
1                                                American            10F   
2                                                   Pizza            10F   
4                                                  Bakery            10F   
5                                         Café/Coffee/Tea            10F   
6       Latin (Cuban

In [160]:
import pymongo



In [134]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.restaurantgradesDB

#collections
groupA = db["GroupA"]
groupB = db["GroupB"]
groupC = db["GroupC"]
groupP = db["GroupP"]